In [1]:
from pathlib import Path
import mido

train_path = Path("data/train")

# Collect all Mozart MIDI files
mozart_midis = []
for midi_file in train_path.glob("*.mid"):
    if "mozart" in midi_file.name.lower():
        try:
            midi_obj = mido.MidiFile(midi_file)
            mozart_midis.append(midi_obj)
        except Exception as e:
            print(f"Could not load {midi_file}: {e}")

print(f"Loaded {len(mozart_midis)} Mozart MIDI files from train/")


Could not load data\train\mozart-piano_sonatas-nueva_carpeta-k281_piano_sonata_n03_3mov.mid: Could not decode key with 2 flats and mode 2
Could not load data\train\unknown_artist-i_o-mozart_k550.mid: MThd not found. Probably not a MIDI file
Loaded 231 Mozart MIDI files from train/


In [5]:
import midi_conversion
import importlib

importlib.reload(midi_conversion)

mozart_texts = []

# print(midi_to_text(mozart_midis[0]))

total = len(mozart_midis)
for i, mid in enumerate(mozart_midis, start=1):
    mozart_texts.append(midi_conversion.midi_to_text(mid, "mozart"))
    print(f"Processed {i}/{total} files", end="\r")

SEQ_SOS = "<SOS>"
SEQ_EOS = "<EOS>"
seqs = [f"{SEQ_SOS} {txt} {SEQ_EOS}" for txt in mozart_texts]

print("Mozart text processing completed.")

Mozart text processing completed.


In [6]:
from collections import Counter
import torch, math
import torch.nn as nn
import torch.nn.functional as F

# 1) tokenize: your format is already space-separated
all_tokens = []
for s in seqs:
    all_tokens.extend(s.split())

vocab = sorted(set(all_tokens))
stoi = {t:i for i, t in enumerate(vocab)}
itos = {i:t for t,i in stoi.items()}
vocab_size = len(vocab)
print("vocab_size:", vocab_size)

def encode(text: str):
    return [stoi[t] for t in text.split()]

def decode(ids):
    return " ".join(itos[int(i)] for i in ids)

# concatenate all pieces into one long stream
ids = torch.tensor([stoi[t] for t in all_tokens], dtype=torch.long)

# train/val split
n = int(0.9 * len(ids))
train_data = ids[:n]
val_data   = ids[n:]


vocab_size: 1114


In [20]:
block_size = 128   # sequence length
batch_size = 32

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size]     for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y


In [17]:
class MozartTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256,
                 n_head=4, n_layer=6, dim_ff=512, block_size=512):
        super().__init__()
        self.block_size = block_size
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size, d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_head,
            dim_feedforward=dim_ff,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layer)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        B, T = x.shape
        assert T <= self.block_size
        pos = torch.arange(T, device=x.device).unsqueeze(0).expand(B, T)
        h = self.tok_emb(x) + self.pos_emb(pos)

        # causal mask: prevent attention to future positions
        mask = torch.triu(torch.ones(T, T, device=x.device) * float("-inf"), diagonal=1)
        h = self.encoder(h, mask)
        logits = self.lm_head(h)
        return logits


In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

model = MozartTransformer(vocab_size, block_size=block_size)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

Device: cuda


In [21]:
def estimate_loss():
    model.eval()
    out = {}
    with torch.no_grad():
        for split in ["train", "val"]:
            losses = []
            correct = 0
            total = 0
            for _ in range(10):
                xb, yb = get_batch(split)
                logits = model(xb)
                loss = F.cross_entropy(
                    logits.view(-1, vocab_size),
                    yb.view(-1)
                )
                losses.append(loss.item())

                # Accuracy
                preds = torch.argmax(logits, dim=-1)
                correct += (preds == yb).float().sum().item()
                total += yb.numel()
                
            avg_loss = sum(losses) / len(losses)
            accuracy = correct / total
            out[split] = (avg_loss, accuracy)
    model.train()
    return out

max_iters = 6000
eval_interval = 250

for step in range(max_iters):
    xb, yb = get_batch("train")
    logits = model(xb)
    loss = F.cross_entropy(
        logits.view(-1, vocab_size),
        yb.view(-1)
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % eval_interval == 0:
        losses = estimate_loss()
        train_loss, train_acc = losses["train"]
        val_loss, val_acc = losses["val"]
        print(f"step {step}: train loss {train_loss:.3f}, acc {train_acc:.3f} | val loss {val_loss:.3f}, acc {val_acc:.3f}")


step 0: train loss 2.187, acc 0.446 | val loss 2.149, acc 0.438
step 250: train loss 2.222, acc 0.436 | val loss 2.057, acc 0.460
step 500: train loss 2.113, acc 0.466 | val loss 2.065, acc 0.464
step 750: train loss 2.128, acc 0.465 | val loss 2.006, acc 0.476
step 1000: train loss 2.125, acc 0.463 | val loss 1.973, acc 0.485
step 1250: train loss 1.996, acc 0.491 | val loss 2.000, acc 0.479
step 1500: train loss 2.066, acc 0.477 | val loss 1.993, acc 0.485
step 1750: train loss 2.075, acc 0.473 | val loss 1.947, acc 0.497
step 2000: train loss 1.984, acc 0.493 | val loss 1.931, acc 0.496
step 2250: train loss 1.905, acc 0.508 | val loss 1.942, acc 0.494
step 2500: train loss 1.908, acc 0.516 | val loss 1.885, acc 0.511
step 2750: train loss 1.969, acc 0.499 | val loss 1.941, acc 0.496
step 3000: train loss 1.872, acc 0.519 | val loss 1.850, acc 0.517
step 3250: train loss 1.825, acc 0.529 | val loss 1.866, acc 0.514
step 3500: train loss 1.904, acc 0.518 | val loss 1.859, acc 0.517
s

In [22]:
SOS_ID = stoi[SEQ_SOS]
EOS_ID = stoi[SEQ_EOS]

@torch.no_grad()
def generate(start_tokens=None, max_new_tokens=200):
    model.eval()
    if start_tokens is None:
        x = torch.tensor([[SOS_ID]], dtype=torch.long, device=device)
    else:
        x = torch.tensor([start_tokens], dtype=torch.long, device=device)

    for _ in range(max_new_tokens):
        x_cond = x[:, -block_size:]
        logits = model(x_cond)
        logits = logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)  # sample
        x = torch.cat([x, next_id], dim=1)

        # stop at EOS
        if int(next_id[0, 0]) == EOS_ID:
            break

    return x[0].tolist()

# Seed with first few tokens from a real piece
seed_tokens = encode(mozart_texts[0])[:50]
generated_ids = generate(seed_tokens, max_new_tokens=800)
generated_text = decode(generated_ids)
print('First 200 chars of generated text:\n')
print(generated_text[:200])

First 200 chars of generated text:

COMPOSER_mozart TICKS_PER_BEAT_480 TIME_SIGNATURE_4/4 KEY_D TEMPO_BPM_36 TEMPO_BPM_37 TEMPO_BPM_38 POS_0 NOTE71_ON POS_0 NOTE66_ON POS_0.042 NOTE71_OFF TEMPO_BPM_37 POS_0.5 NOTE74_ON POS_0.525 NOTE66_


In [24]:
from midi_conversion import text_to_midi
import importlib
import os

importlib.reload(midi_conversion)

mid = text_to_midi(generated_text)

# Create output directory if it doesn't exist
os.makedirs("generated", exist_ok=True)

# Save to path
output_path = os.path.join("generated", "mozart_output.mid")
mid.save(output_path)

In [72]:
# TESTING TO SEE IF TEXT_TO_MIDI WORKS

test_text = "TICKS_PER_BEAT_480 TIME_SIGNATURE_4/4 KEY_D POS_0 NOTE71_ON POS_0 NOTE66_ON POS_0.042 NOTE71_OFF POS_0.5 NOTE74_ON POS_0.525 NOTE66_OFF POS_0.933 NOTE74_OFF POS_0 NOTE74_ON NOTE68_ON NOTE59_ON NOTE53_ON POS_0 NOTE73_ON NOTE61_ON POS_0.025 NOTE74_OFF POS_0.033 NOTE59_OFF POS_0.5 NOTE71_ON NOTE62_ON POS_0.525 NOTE73_OFF NOTE61_OFF POS_0.925 NOTE68_OFF POS_0.933 NOTE71_OFF NOTE62_OFF POS_0.983 NOTE53_OFF POS_0 NOTE66_ON NOTE71_ON NOTE62_ON NOTE54_ON POS_0.875 NOTE71_OFF NOTE66_OFF POS_0.967 NOTE62_OFF POS_0 NOTE70_ON NOTE61_ON POS_0.375 NOTE76_ON POS_0.396 NOTE70_OFF POS_0.483 NOTE76_OFF NOTE54_OFF POS_0.492 NOTE61_OFF POS_0.5 NOTE73_ON NOTE67_ON NOTE64_ON POS_0.875 NOTE70_ON POS_0.9 NOTE73_OFF POS_0.958 NOTE64_OFF POS_0.975 NOTE67_OFF POS_0.983 NOTE70_OFF POS_0 NOTE73_ON NOTE66_ON NOTE62_ON POS_0.25 NOTE71_ON POS_0.275 NOTE73_OFF POS_0.442 NOTE71_OFF POS_0.45 NOTE66_OFF POS_0.458 NOTE62_OFF POS_0.5 NOTE74_ON NOTE66_ON NOTE58_ON POS_0.75 NOTE73_ON POS_0.775 NOTE74_OFF POS_0.944 NOTE73_OFF POS_0.958 NOTE58_OFF POS_0.967 NOTE66_OFF POS_0 NOTE76_ON NOTE66_ON NOTE59_ON POS_0.25 NOTE74_ON POS_0.279 NOTE76_OFF POS_0.433 NOTE59_OFF NOTE66_OFF POS_0.44 NOTE74_OFF POS_0 NOTE62_ON NOTE66_ON POS_0.217 NOTE66_OFF NOTE62_OFF POS_0.25 NOTE62_ON NOTE66_ON POS_0.467 NOTE66_OFF NOTE62_OFF POS_0.5 NOTE71_ON NOTE83_ON NOTE62_ON NOTE66_ON POS_0.717 NOTE66_OFF NOTE62_OFF POS_0.75 NOTE62_ON NOTE66_ON POS_0.967 NOTE66_OFF NOTE62_OFF POS_0 NOTE61_ON NOTE64_ON POS_0.217 NOTE64_OFF NOTE61_OFF POS_0.25 NOTE61_ON NOTE64_ON POS_0.467 NOTE64_OFF NOTE61_OFF POS_0.5 NOTE70_ON NOTE82_ON NOTE61_ON NOTE64_ON POS_0.544 NOTE71_OFF POS_0.546 NOTE83_OFF POS_0.717 NOTE64_OFF NOTE61_OFF POS_0.75 NOTE61_ON NOTE64_ON POS_0.967 NOTE64_OFF NOTE61_OFF POS_0 NOTE70_OFF NOTE59_ON NOTE62_ON POS_0.008 NOTE82_OFF POS_0.217 NOTE62_OFF NOTE59_OFF POS_0.25 NOTE59_ON NOTE62_ON POS_0.467 NOTE62_OFF NOTE59_OFF POS_0.5 NOTE79_ON NOTE59_ON NOTE62_ON POS_0.717 NOTE62_OFF NOTE59_OFF POS_0.75 NOTE59_ON NOTE62_ON POS_0.875 NOTE81_ON POS_0.9 NOTE79_OFF POS_0.967 NOTE62_OFF NOTE59_OFF POS_0 NOTE79_ON NOTE58_ON NOTE61_ON POS_0.019 NOTE81_OFF POS_0.217 NOTE61_OFF NOTE58_OFF POS_0.25 NOTE58_ON NOTE61_ON POS_0.467 NOTE61_OFF NOTE58_OFF POS_0.5 NOTE78_ON NOTE58_ON NOTE61_ON POS_0.533 NOTE79_OFF POS_0.717 NOTE61_OFF NOTE58_OFF POS_0.75 NOTE58_ON NOTE61_ON POS_0.967 NOTE61_OFF NOTE58_OFF POS_0 NOTE78_OFF NOTE59_ON NOTE62_ON POS_0.217 NOTE62_OFF NOTE59_OFF POS_0.25 NOTE59_ON NOTE62_ON POS_0.467 NOTE62_OFF NOTE59_OFF POS_0.5 NOTE67_ON NOTE79_ON NOTE59_ON NOTE62_ON POS_0.717 NOTE62_OFF NOTE59_OFF POS_0.75 NOTE59_ON NOTE62_ON POS_0.967 NOTE62_OFF NOTE59_OFF POS_0 NOTE57_ON NOTE61_ON POS_0.217 NOTE61_OFF NOTE57_OFF POS_0.25 NOTE57_ON NOTE61_ON POS_0.467 NOTE61_OFF NOTE57_OFF POS_0.5 NOTE66_ON NOTE78_ON NOTE57_ON NOTE61_ON POS_0.544 NOTE79_OFF NOTE67_OFF POS_0.717 NOTE61_OFF NOTE57_OFF POS_0.75 NOTE57_ON NOTE61_ON POS_0.967 NOTE61_OFF NOTE57_OFF POS_0 NOTE66_OFF NOTE78_OFF NOTE55_ON NOTE59_ON POS_0.217 NOTE59_OFF NOTE55_OFF POS_0.25 NOTE55_ON NOTE59_ON POS_0.467 NOTE59_OFF NOTE55_OFF POS_0.5 NOTE76_ON NOTE55_ON NOTE59_ON POS_0.717 NOTE59_OFF NOTE55_OFF POS_0.75 NOTE55_ON NOTE59_ON POS_0.875 NOTE78_ON POS_0.9 NOTE76_OFF POS_0.967 NOTE59_OFF NOTE55_OFF POS_0 NOTE76_ON NOTE57_ON NOTE54_ON POS_0.025 NOTE78_OFF POS_0.25 NOTE59_ON POS_0.275 NOTE57_OFF POS_0.5 NOTE75_ON NOTE60_ON POS_0.525 NOTE59_OFF POS_0.533 NOTE76_OFF POS_0.75 NOTE59_ON POS_0.773 NOTE60_OFF POS_0.933 NOTE75_OFF POS_0.95 NOTE54_OFF POS_0.967 NOTE59_OFF POS_0 NOTE76_ON NOTE59_ON NOTE55_ON POS_0.25 NOTE83_ON POS_0.273 NOTE76_OFF POS_0.5 NOTE81_ON POS_0.527 NOTE83_OFF POS_0.625 NOTE79_ON POS_0.648 NOTE81_OFF POS_0.75 NOTE78_ON POS_0.773 NOTE79_OFF POS_0.875 NOTE76_ON POS_0.9 NOTE78_OFF POS_0.958 NOTE76_OFF POS_0.975 NOTE55_OFF NOTE59_OFF POS_0 NOTE76_ON NOTE57_ON NOTE54_ON POS_0.25 NOTE59_ON POS_0.283 NOTE57_OFF POS_0.5 NOTE75_ON NOTE60_ON POS_0.533 NOTE76_OFF NOTE59_OFF POS_0.75 NOTE59_ON POS_0.783 NOTE60_OFF POS_0.933 NOTE75_OFF POS_0.95 NOTE54_OFF POS_0.967 NOTE59_OFF POS_0 NOTE76_ON NOTE59_ON NOTE55_ON POS_0.25 NOTE83_ON POS_0.275 NOTE76_OFF POS_0.5 NOTE84_ON POS_0.525 NOTE83_OFF POS_0.625 NOTE83_ON POS_0.644 NOTE84_OFF POS_0.75 NOTE81_ON POS_0.773 NOTE83_OFF POS_0.875 NOTE79_ON POS_0.898 NOTE81_OFF POS_0.942 NOTE55_OFF NOTE59_OFF POS_0.975 NOTE79_OFF POS_0 NOTE78_ON NOTE59_ON NOTE56_ON POS_0.25 NOTE61_ON POS_0.275 NOTE59_OFF POS_0.5 NOTE77_ON NOTE62_ON POS_0.517 NOTE61_OFF POS_0.525 NOTE78_OFF POS_0.75 NOTE61_ON POS_0.775 NOTE62_OFF POS_0.875 NOTE56_OFF POS_0.933 NOTE77_OFF POS_0.967 NOTE61_OFF POS_0 NOTE78_ON NOTE61_ON NOTE58_ON POS_0.375 NOTE73_ON POS_0.381 NOTE78_OFF POS_0.433 NOTE61_OFF POS_0.483 NOTE73_OFF POS_0.5 NOTE73_ON NOTE76_ON NOTE67_ON POS_0.967 NOTE58_OFF POS_0 NOTE66_ON NOTE59_ON POS_0.033 NOTE67_OFF POS_0.5 NOTE71_ON NOTE74_ON NOTE65_ON POS_0.525 NOTE66_OFF POS_0.533 NOTE76_OFF NOTE73_OFF POS_0.933 NOTE71_OFF NOTE74_OFF NOTE65_OFF NOTE59_OFF POS_0 NOTE74_ON NOTE78_ON NOTE66_ON NOTE54_ON POS_0.75 NOTE73_ON NOTE76_ON POS_0.773 NOTE78_OFF POS_0.779 NOTE74_OFF POS_0.875 NOTE71_ON NOTE74_ON POS_0.898 NOTE76_OFF POS_0.9 NOTE73_OFF POS_0 NOTE70_ON NOTE73_ON NOTE54_OFF NOTE66_OFF POS_0.023 NOTE71_OFF POS_0.025 NOTE74_OFF POS_0.375 NOTE70_OFF NOTE73_OFF POS_0 NOTE47_ON POS_0 NOTE42_ON POS_0.033 NOTE47_OFF POS_0.5 NOTE50_ON POS_0.525 NOTE42_OFF POS_0.933 NOTE50_OFF POS_0 NOTE77_ON NOTE68_ON NOTE50_ON POS_0 NOTE76_ON NOTE49_ON POS_0.025 NOTE50_OFF POS_0.033 NOTE77_OFF POS_0.5 NOTE74_ON NOTE47_ON POS_0.525 NOTE49_OFF POS_0.533 NOTE76_OFF POS_0.925 NOTE68_OFF POS_0.933 NOTE74_OFF NOTE47_OFF POS_0 NOTE68_ON NOTE74_ON NOTE45_ON POS_0.25 NOTE69_ON NOTE73_ON POS_0.275 NOTE74_OFF NOTE68_OFF POS_0.433 NOTE45_OFF POS_0.467 NOTE69_OFF NOTE73_OFF POS_0.5 NOTE45_ON POS_0.75 NOTE67_ON NOTE71_ON POS_0.933 NOTE45_OFF POS_0.967 NOTE71_OFF NOTE67_OFF POS_0 NOTE47_ON POS_0.25 NOTE66_ON NOTE69_ON POS_0.433 NOTE47_OFF POS_0.467 NOTE69_OFF NOTE66_OFF POS_0.5 NOTE49_ON POS_0.75 NOTE64_ON NOTE67_ON POS_0.933 NOTE49_OFF POS_0.967 NOTE67_OFF NOTE64_OFF POS_0 NOTE66_ON NOTE69_ON NOTE50_ON POS_0.125 NOTE64_ON NOTE67_ON POS_0.158 NOTE69_OFF NOTE66_OFF POS_0.25 NOTE62_ON NOTE66_ON POS_0.283 NOTE64_OFF NOTE67_OFF POS_0.375 NOTE64_ON NOTE67_ON POS_0.383 NOTE66_OFF POS_0.408 NOTE62_OFF POS_0.433 NOTE50_OFF POS_0.5 NOTE62_ON NOTE66_ON POS_0.525 NOTE67_OFF POS_0.533 NOTE64_OFF POS_0.717 NOTE62_OFF NOTE66_OFF POS_0.75 NOTE62_ON POS_0 NOTE61_ON POS_0.033 NOTE62_OFF POS_0.25 NOTE62_ON POS_0.275 NOTE61_OFF POS_0.5 NOTE57_ON POS_0.525 NOTE62_OFF POS_0.75 NOTE54_ON POS_0.775 NOTE57_OFF POS_0.967 NOTE54_OFF POS_0 NOTE55_ON NOTE58_ON POS_0.217 NOTE58_OFF NOTE55_OFF POS_0.25 NOTE55_ON NOTE58_ON POS_0.467 NOTE58_OFF NOTE55_OFF POS_0.5 NOTE61_ON NOTE73_ON NOTE55_ON NOTE58_ON POS_0.717 NOTE58_OFF NOTE55_OFF POS_0.75 NOTE55_ON NOTE58_ON POS_0.967 NOTE58_OFF NOTE55_OFF POS_0 NOTE54_ON NOTE57_ON POS_0.217 NOTE57_OFF NOTE54_OFF POS_0.25 NOTE54_ON NOTE57_ON POS_0.467 NOTE57_OFF NOTE54_OFF POS_0.5 NOTE62_ON NOTE74_ON NOTE54_ON NOTE57_ON POS_0.525 NOTE73_OFF POS_0.533 NOTE61_OFF POS_0.717 NOTE57_OFF NOTE54_OFF POS_0.75 NOTE54_ON NOTE57_ON POS_0.933 NOTE62_OFF NOTE74_OFF POS_0.967 NOTE57_OFF NOTE54_OFF POS_0 NOTE55_ON NOTE64_ON POS_0.217 NOTE64_OFF NOTE55_OFF POS_0.25 NOTE55_ON NOTE64_ON POS_0.467 NOTE64_OFF NOTE55_OFF POS_0.5 NOTE73_ON NOTE85_ON NOTE55_ON NOTE64_ON POS_0.717 NOTE64_OFF NOTE55_OFF POS_0.75 NOTE55_ON NOTE64_ON POS_0.967 NOTE64_OFF NOTE55_OFF POS_0 NOTE56_ON NOTE65_ON POS_0.217 NOTE65_OFF NOTE56_OFF POS_0.25 NOTE56_ON NOTE65_ON POS_0.458 NOTE85_OFF POS_0.467 NOTE73_OFF NOTE65_OFF NOTE56_OFF POS_0.5 NOTE74_ON NOTE86_ON NOTE56_ON NOTE65_ON POS_0.717 NOTE65_OFF NOTE56_OFF POS_0.75 NOTE83_ON NOTE56_ON NOTE62_ON POS_0.783 NOTE86_OFF POS_0.967 NOTE62_OFF NOTE56_OFF POS_0 NOTE74_OFF NOTE81_ON NOTE57_ON NOTE61_ON POS_0.033 NOTE83_OFF POS_0.217 NOTE61_OFF NOTE57_OFF POS_0.25 NOTE57_ON NOTE61_ON POS_0.467 NOTE61_OFF NOTE57_OFF POS_0.5 NOTE80_ON NOTE59_ON NOTE62_ON POS_0.533 NOTE81_OFF POS_0.717 NOTE62_OFF NOTE59_OFF POS_0.75 NOTE59_ON NOTE62_ON POS_0.933 NOTE80_OFF POS_0.967 NOTE62_OFF NOTE59_OFF POS_0 NOTE79_ON NOTE61_ON NOTE64_ON POS_0.217 NOTE64_OFF NOTE61_OFF POS_0.25 NOTE61_ON NOTE64_ON POS_0.467 NOTE64_OFF NOTE61_OFF POS_0.5 NOTE78_ON NOTE62_ON NOTE69_ON POS_0.533 NOTE79_OFF POS_0.625 NOTE79_ON POS_0.652 NOTE78_OFF POS_0.717 NOTE69_OFF NOTE62_OFF POS_0.75 NOTE80_ON NOTE62_ON NOTE66_ON POS_0.771 NOTE79_OFF POS_0.875 NOTE81_ON POS_0.9 NOTE80_OFF POS_0.967 NOTE66_OFF NOTE62_OFF POS_0.975 NOTE81_OFF POS_0 NOTE81_ON NOTE66_ON NOTE57_ON POS_0.65 NOTE66_OFF POS_0.75 NOTE78_ON NOTE62_ON POS_0.775 NOTE81_OFF POS_0.875 NOTE57_OFF POS_0.95 NOTE78_OFF POS_0.967 NOTE62_OFF POS_0 NOTE76_ON NOTE61_ON NOTE57_ON POS_0.396 NOTE76_OFF POS_0.433 NOTE57_OFF NOTE61_OFF POS_0 NOTE66_ON NOTE69_ON NOTE38_ON POS_0.217 NOTE69_OFF NOTE66_OFF POS_0.25 NOTE66_ON NOTE69_ON POS_0.467 NOTE69_OFF NOTE66_OFF POS_0.5 NOTE66_ON NOTE69_ON POS_0.717 NOTE69_OFF NOTE66_OFF POS_0.75 NOTE66_ON NOTE69_ON POS_0.875 NOTE42_ON POS_0.908 NOTE38_OFF POS_0.933 NOTE45_ON POS_0.963 NOTE42_OFF POS_0.967 NOTE69_OFF NOTE66_OFF POS_0 NOTE66_ON NOTE69_ON NOTE50_ON POS_0.033 NOTE45_OFF POS_0.217 NOTE69_OFF NOTE66_OFF POS_0.25 NOTE66_ON NOTE69_ON POS_0.467 NOTE69_OFF NOTE66_OFF POS_0.496 NOTE50_OFF POS_0.5 NOTE66_ON NOTE69_ON POS_0.717 NOTE69_OFF NOTE66_OFF POS_0.75 NOTE66_ON NOTE69_ON POS_0.967 NOTE69_OFF NOTE66_OFF"

mid = text_to_midi(test_text)

# Create output directory if it doesn't exist
os.makedirs("generated", exist_ok=True)

# Save to path
output_path = os.path.join("generated", "mozart_test_output.mid")
mid.save(output_path)